In [ ]:
!pip install bs4
!pip install selenium
!pip install pandas
!pip install tqdm

In [ ]:
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from os import environ
from selenium.webdriver import Remote, ChromeOptions
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chromium.remote_connection import ChromiumRemoteConnection
import urllib
import time
import csv
import warnings

warnings.filterwarnings("ignore")

In [ ]:
AUTH = 'brd-customer-hl_9b917a04-zone-tom_bijaoui_scraping:qdun9u7lgdd9'
SBR_WEBDRIVER = f'https://{AUTH}@brd.superproxy.io:9515'

In [ ]:
scraping_companies_websites = pd.read_csv('/content/scrapping_companies_comparably.csv')
jeremy_df = scraping_companies_websites.iloc[0:401]
jeremy_df.to_csv('/content/jeremy_scraping_df.csv')
scraping_companies_websites = scraping_companies_websites.iloc[401:]

In [ ]:
if __name__ == '__main__':

    companies_list = scraping_companies_websites['company_name'].tolist()

    with open('scraped_comparably.csv', mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['name', 'scraped_text'])

    uncompleted_url = 'https://www.comparably.com/search?q='

    for i in tqdm(range(len(companies_list)), desc='Scraping...'):
        name = companies_list[i]
        sbr_connection = ChromiumRemoteConnection(SBR_WEBDRIVER, 'goog', 'chrome')
        driver = Remote(sbr_connection, options=ChromeOptions())

        try:
            driver.get(uncompleted_url + urllib.parse.quote_plus(name))

        except Exception as e:
            continue
        time.sleep(7)

        likely_results_containers = BeautifulSoup(driver.page_source, 'html.parser').find('div', class_='results gs-row offset')

        if not likely_results_containers:
            continue

        best_result_a = likely_results_containers.find_next('a')
        best_result_url = best_result_a.get('href')
        best_result_title = likely_results_containers.find('div', class_='resultsCard-logo-title').get_text()
        if best_result_title == 'Fox Corporation':
            continue

        driver.quit()

        sbr_connection = ChromiumRemoteConnection(SBR_WEBDRIVER, 'goog', 'chrome')
        driver = Remote(sbr_connection, options=ChromeOptions())

        try:
            driver.get(best_result_url)

        except Exception as e:
            continue

        time.sleep(7)

        try:
            core_values_containers = BeautifulSoup(driver.page_source, 'html.parser').find('div', class_='scrollTarget divider')
            core_values_p = core_values_containers.find_all('p', class_='section-text')

            scraped_text = ''
            for p in core_values_p:
                scraped_text += p.get_text()

            if scraped_text == '':
                scraped_text = None

            with open('scraped_comparably.csv', mode='a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerow([name, scraped_text])

            driver.quit()

        except Exception as e:
            continue

Scraping...:  54%|█████▍    | 252/463 [2:19:51<1:57:06, 33.30s/it]


WebDriverException: Message: Internal Server Error
